## Mental health detection

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from transformers import pipeline, Trainer, TrainingArguments, BertTokenizer, BertForSequenceClassification

/Users/venessa/Desktop/SchemesSG_v3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Download the fine-tuned models from here into the same folder:
https://drive.google.com/drive/folders/1XhWWajqF6tI_9wfRp-kI18HQcgObxTez?usp=sharing

### Data loading and preparation

In [ ]:
# Load labelled data
df = pd.read_csv('data_labelled.csv')

# Train-test split
X = df['query']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define evaluator function for results

def evaluate_results(y, y_pred):
    print(classification_report(y, y_pred))
    print('Confusion Matrix:')
    print(confusion_matrix(y, y_pred))
    print('Accuracy:', accuracy_score(y, y_pred))


### Evaluation of 3 fine-tuned models on test dataset

Model #1 performs the best in terms of overall accuracy and F1-score, especially for positive examples.

In [113]:
# predict using model 1
classifier = pipeline('text-classification', model='ardavey/bert-large-depression-classification-model')
y_pred = classifier(X_test.tolist())
y_pred = [1 if pred['label'] == 'LABEL_1' else 0 for pred in y_pred]
evaluate_results(y_test, y_pred)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


              precision    recall  f1-score   support

           0       0.87      0.96      0.91       189
           1       0.60      0.30      0.40        40

    accuracy                           0.84       229
   macro avg       0.73      0.63      0.65       229
weighted avg       0.82      0.84      0.82       229

Confusion Matrix:
[[181   8]
 [ 28  12]]
Accuracy: 0.8427947598253275


In [110]:
# predict using model 2
classifier = pipeline("text-classification", model="rafalposwiata/deproberta-large-depression")
y_pred = classifier(X_test.tolist())
y_pred = [1 if pred['label'] == 'LABEL_1' else 0 for pred in y_pred]
evaluate_results(y_test, y_pred)

/Users/venessa/Desktop/SchemesSG_v3/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


              precision    recall  f1-score   support

           0       0.83      1.00      0.90       189
           1       0.00      0.00      0.00        40

    accuracy                           0.83       229
   macro avg       0.41      0.50      0.45       229
weighted avg       0.68      0.83      0.75       229

Confusion Matrix:
[[189   0]
 [ 40   0]]
Accuracy: 0.8253275109170306


/Users/venessa/Desktop/SchemesSG_v3/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/venessa/Desktop/SchemesSG_v3/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/venessa/Desktop/SchemesSG_v3/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [112]:
# predict using model 3
classifier = pipeline("text-classification", model="poudel/Depression_and_Non-Depression_Classifier")     
y_pred = classifier(X_test.tolist())
y_pred = [1 if pred['label'] == 'LABEL_1' else 0 for pred in y_pred]
evaluate_results(y_test, y_pred)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


              precision    recall  f1-score   support

           0       0.79      0.76      0.78       189
           1       0.04      0.05      0.05        40

    accuracy                           0.64       229
   macro avg       0.42      0.41      0.41       229
weighted avg       0.66      0.64      0.65       229

Confusion Matrix:
[[144  45]
 [ 38   2]]
Accuracy: 0.6375545851528385


### Further fine-tuning of model

Fine-tuning can be done in two ways: updating all model parameters (without freezing) or only updating the classification head (with freezing).

- **Without Freezing**: The entire BERT model, including the pre-trained layers, is updated during fine-tuning. This enables the model to adapt its internal representations to the specific dataset. However, it requires more computational resources and has a higher risk of overfitting when the dataset is small.

- **With Freezing**: The pre-trained BERT layers are kept fixed, and only the classification head (the top layer) is updated during fine-tuning. This approach reduces computational cost, speeds up training, and lowers the risk of overfitting, as fewer parameters are updated.

In [ ]:
# Tokenize the data using BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

def tokenize_function(examples):
    return tokenizer(examples['query'], padding=True, truncation=True)

# Create a Dataset object for Hugging Face
train_dataset = Dataset.from_pandas(pd.DataFrame({'query': X_train, 'label': y_train}))
test_dataset = Dataset.from_pandas(pd.DataFrame({'query': X_test, 'label': y_test}))

# Tokenize the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

In [ ]:
# Fine-tuning BERT model without freezing

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('ardavey/bert-large-depression-classification-model', num_labels=2)

# Set up the Trainer
training_args = TrainingArguments(
    output_dir='.',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.predictions.argmax(axis=-1), p.label_ids)}
)

#  Fine-tune the model
trainer.train()

# Evaluate the model after fine-tuning
predictions, labels, metrics = trainer.predict(test_dataset)

# Convert predictions to binary labels
predicted_labels = predictions.argmax(axis=-1)

# Step 8: Print evaluation metrics
evaluate_results(y_test, predicted_labels)

# Save the fine-tuned model for future use
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

/Users/venessa/Desktop/SchemesSG_v3/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 229/229 [00:00<00:00, 12454.56 examples/s]
/Users/venessa/Desktop/SchemesSG_v3/.venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  3%|▎         | 10/342 [00:08<02:31,  2.20it/s]

{'loss': 0.7609, 'grad_norm': 6.425285339355469, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}


  6%|▌         | 20/342 [00:12<02:08,  2.51it/s]

{'loss': 0.3403, 'grad_norm': 0.18243823945522308, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.18}


  9%|▉         | 30/342 [00:16<02:04,  2.51it/s]

...

                                                 
100%|██████████| 342/342 [02:27<00:00,  2.31it/s]


{'eval_loss': 0.36817458271980286, 'eval_accuracy': 0.9344978165938864, 'eval_runtime': 1.1718, 'eval_samples_per_second': 195.432, 'eval_steps_per_second': 12.801, 'epoch': 3.0}
{'train_runtime': 147.9297, 'train_samples_per_second': 18.495, 'train_steps_per_second': 2.312, 'train_loss': 0.37939673118097217, 'epoch': 3.0}


100%|██████████| 15/15 [00:01<00:00, 14.05it/s]


              precision    recall  f1-score   support

           0       0.94      0.98      0.96       189
           1       0.90      0.70      0.79        40

    accuracy                           0.93       229
   macro avg       0.92      0.84      0.87       229
weighted avg       0.93      0.93      0.93       229

Confusion Matrix:
[[186   3]
 [ 12  28]]
Accuracy: 0.9344978165938864


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json')

In [ ]:
# Fine-tune model, freezing BERT layers

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('ardavey/bert-large-depression-classification-model', num_labels=2)

# Freeze the BERT layers
for param in model.bert.parameters():
    param.requires_grad = False

# Set up the Trainer
training_args = TrainingArguments(
    output_dir='.',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.predictions.argmax(axis=-1), p.label_ids)}
)

#  Fine-tune the model
trainer.train()

# Evaluate the model after fine-tuning
predictions, labels, metrics = trainer.predict(test_dataset)

# Convert predictions to binary labels
predicted_labels = predictions.argmax(axis=-1)

# Step 8: Print evaluation metrics
evaluate_results(y_test, predicted_labels)

# Save the fine-tuned model for future use
model.save_pretrained('./fine_tuned_model_freeze')
tokenizer.save_pretrained('./fine_tuned_model_freeze')

  3%|▎         | 11/342 [00:01<00:31, 10.37it/s]

{'loss': 0.7636, 'grad_norm': 6.137767791748047, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.09}


  6%|▌         | 21/342 [00:02<00:30, 10.48it/s]

{'loss': 0.3415, 'grad_norm': 0.14921130239963531, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.18}


  9%|▉         | 31/342 [00:02<00:29, 10.49it/s]

...

100%|██████████| 342/342 [00:38<00:00,  8.84it/s]


{'train_runtime': 38.6606, 'train_samples_per_second': 70.77, 'train_steps_per_second': 8.846, 'train_loss': 0.6099620074556585, 'epoch': 3.0}


100%|██████████| 15/15 [00:01<00:00, 14.31it/s]


              precision    recall  f1-score   support

           0       0.87      0.96      0.91       189
           1       0.60      0.30      0.40        40

    accuracy                           0.84       229
   macro avg       0.73      0.63      0.65       229
weighted avg       0.82      0.84      0.82       229

Confusion Matrix:
[[181   8]
 [ 28  12]]
Accuracy: 0.8427947598253275


('./fine_tuned_model_freeze/tokenizer_config.json',
 './fine_tuned_model_freeze/special_tokens_map.json',
 './fine_tuned_model_freeze/vocab.txt',
 './fine_tuned_model_freeze/added_tokens.json')

Fine-tuning without freezing led to greater improvements in both overall accuracy and F1 score compared to fine-tuning with freezing.


### Next steps

For next steps we can either:
1. Continue to improve the model by exploring more models, synthesizing more data, etc
2. Integrate the model with the existing code to enable mental health detection functionality

In [3]:
# Test the fine-tuned model

classifier = pipeline('text-classification', model='./fine_tuned_model')
prediction = classifier('I feel sad and hopeless')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
